In [20]:
import pandas as pd
import logging
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
print("Passed")

Passed


In [21]:
from torch import cuda
device = "cuda" if cuda.is_available() else "cpu"
print("passed")

passed


Start to interpret the data

In [22]:
#data extraction

train = pd.read_csv("/kaggle/input/motion/motionStrike_TVcodes_data.tsv", delimiter='\t' )
train.shape

(13803, 22)

In [23]:
train.head()

,CaseReferenceNumber,MotionID,CaseLocation,CaseMajorCode,CaseMinorCode,CaseTrialListType,CaseMarkingCode,CaseDispositionJudgeJurisNo,CaseDispositionDocketLegendCode,MotionResultCode,...,MotionJurisNumber,MotionDocumentPriorityCode,CaseAttorneyJuris,CaseAttorneyType,MotionTimeDuration,MotionDocumentTypeName,SelfRepBeforeMotionFileCount,SelfRepBeforeMotionDecidedCount,SelfRepBeforeMotionFileBool,SelfRepBeforeMotionDecidedBool
0,207,1751,FBT,V,1,9,16.0,416586.0,JDGDACT,GR,...,418027.0,30,418027.0,J,78.0,MOTION TO STRIKE,1,1,True,True
1,1472,10915,FBT,V,1,9,16.0,NaN,WDACT,DN,...,416586.0,30,416586.0,R,22.0,MOTION TO STRIKE,0,0,False,False
2,1494,11050,NNI,V,1,9,16.0,NaN,WDACT,GR,...,422392.0,30,422392.0,J,285.0,MOTION TO STRIKE,0,0,False,False
3,371,2708,LLI,T,90,-1,NaN,418028.0,SJD,OR,...,418028.0,30,418028.0,R,120.0,MOTION TO STRIKE,0,0,False,False
4,1422,10373,HHD,T,2,-1,NaN,422394.0,SJ,GR,...,81181.0,30,81181.0,R,153.0,MOTION TO STRIKE,0,0,False,False


feature engineering for attorney specialization 

In [26]:
#dirchlet multinomial distribution
from scipy.stats import entropy
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import math
from multiprocessing import Pool
import random

#create isolated dataset for attorny calc 
specialtab = train[["CaseAttorneyJuris", "CaseMajorCode"]]
specialtab.head()
    

,CaseAttorneyJuris,CaseMajorCode
0,418027.0,V
1,416586.0,V
2,422392.0,V
3,418028.0,T
4,81181.0,T


In [27]:

def calculate_attorney_specialization(specialtab):
    # Make a copy of the input dataframe to avoid the warning
    df = specialtab.copy()
    
    # Group by attorney and count case types
    case_counts = pd.crosstab(df['CaseAttorneyJuris'], 
                             df['CaseMajorCode'])
    
    # Calculate entropy with Dirichlet smoothing for each attorney
    attorney_entropy = {}
    for attorney in case_counts.index:
        counts = case_counts.loc[attorney]
        
        # Apply Dirichlet smoothing (adding 1 to each count)
        smoothed_counts = counts + 1
        # Calculate proportions
        proportions = smoothed_counts / smoothed_counts.sum()
        # Calculate entropy
        attorney_entropy[attorney] = entropy(proportions, base=2)
    
    # Add entropy back to dataframe
    df['AttorneySpecialization'] = df['CaseAttorneyJuris'].map(attorney_entropy)
    
    return df
print("function passed")


function passed


test the above code

In [28]:
# Use the function
specialentropy = calculate_attorney_specialization(specialtab)

# Look at entropy scores
print("\nEntropy scores:")
print(specialentropy.groupby('CaseAttorneyJuris')['AttorneySpecialization'].head())




    


Entropy scores:
0        0.999273
1        0.992774
2        0.999571
3        0.902393
4        0.977001
           ...   
13730    0.954434
13752    0.985228
13760    0.954434
13768    0.985228
13776    0.985228
Name: AttorneySpecialization, Length: 1276, dtype: float64


In [29]:
#To see most specialized attorneys
print("\nMost specialized attorneys (lowest entropy):")
print(specialentropy.groupby('CaseAttorneyJuris')
      .agg({'AttorneySpecialization': 'first', 'CaseMajorCode': lambda x: x.value_counts().to_dict()})
      .sort_values('AttorneySpecialization')
      .head())


Most specialized attorneys (lowest entropy):
                   AttorneySpecialization      CaseMajorCode
CaseAttorneyJuris                                           
407897.0                         0.286397          {'T': 18}
412189.0                         0.391244          {'T': 11}
435703.0                         0.413817          {'T': 10}
80808.0                          0.413817          {'T': 10}
81240.0                          0.422001  {'T': 31, 'V': 2}


New trained database:


In [31]:
specialentropy.head()


,CaseAttorneyJuris,CaseMajorCode,AttorneySpecialization
0,418027.0,V,0.999273
1,416586.0,V,0.992774
2,422392.0,V,0.999571
3,418028.0,T,0.902393
4,81181.0,T,0.977001


In [32]:
legal = train[["CaseLocation","CaseMajorCode","MotionJurisNumber","MotionResultCode"]]
print("passed")

passed


In [33]:
legal.head()

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode
0,FBT,V,418027.0,GR
1,FBT,V,416586.0,DN
2,NNI,V,422392.0,GR
3,LLI,T,418028.0,OR
4,HHD,T,81181.0,GR


combine data sets together

In [34]:
# we need to add Attorney specialization to the legal data group for easier training in the future
legal = legal.assign(AttorneySpecialization=specialentropy["AttorneySpecialization"])
legal.head()

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode,AttorneySpecialization
0,FBT,V,418027.0,GR,0.999273
1,FBT,V,416586.0,DN,0.992774
2,NNI,V,422392.0,GR,0.999571
3,LLI,T,418028.0,OR,0.902393
4,HHD,T,81181.0,GR,0.977001


We need to split the data into two groups, one for training and one for testing, we are trying to use these data values to predict the outcome of MotionResultCode

We only want granted or denied motion codes so we need to filter everything but GR,DN,DS

In [35]:
filtered = legal[legal["MotionResultCode"].isin(["GR","DN"])]
print("passed")

passed


In [36]:
filtered.head()

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode,AttorneySpecialization
0,FBT,V,418027.0,GR,0.999273
1,FBT,V,416586.0,DN,0.992774
2,NNI,V,422392.0,GR,0.999571
4,HHD,T,81181.0,GR,0.977001
6,MMX,T,403965.0,DN,0.976310


we need to create embedding and start to represent word variables as numbers for training

In [37]:
#apply one hot to MotionResultCode
#make this a function later
one_hot = pd.get_dummies(filtered["MotionResultCode"])
one_hot = one_hot.drop("DN", axis = 1)
one_hot.rename(columns={"GR" : "MotionResultCode"}, inplace=True)
one_hot = one_hot.astype(int)
one_hot.head()

#Motion Result Code == 1 means that Motion is GRANTED
#Motion Result Code == 0 means that Motion is DENIED

,MotionResultCode
0,1
1,0
2,1
4,1
6,0


In [38]:
#add one_hot back to the orginal data set filtered and remove current motionresultcode
filtered = filtered.assign(MotionResultCode=one_hot["MotionResultCode"])
filtered

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode,AttorneySpecialization
0,FBT,V,418027.0,1,0.999273
1,FBT,V,416586.0,0,0.992774
2,NNI,V,422392.0,1,0.999571
4,HHD,T,81181.0,1,0.977001
6,MMX,T,403965.0,0,0.976310
...,...,...,...,...,...
13796,FBT,T,413629.0,0,0.998296
13797,NNH,V,421279.0,0,0.967387
13799,HHB,T,431662.0,0,0.884115
13800,TTD,T,436946.0,1,0.991076


In [39]:
#Beta Bernouli for CaseMajorCode
#Tort ==1 Vehicular == 0 
#turn into function
filtered['CaseMajorCode'] = filtered['CaseMajorCode'].apply(lambda x: 1 if x == 'T' else 0)


In [40]:
filtered

,CaseLocation,CaseMajorCode,MotionJurisNumber,MotionResultCode,AttorneySpecialization
0,FBT,0,418027.0,1,0.999273
1,FBT,0,416586.0,0,0.992774
2,NNI,0,422392.0,1,0.999571
4,HHD,1,81181.0,1,0.977001
6,MMX,1,403965.0,0,0.976310
...,...,...,...,...,...
13796,FBT,1,413629.0,0,0.998296
13797,NNH,0,421279.0,0,0.967387
13799,HHB,1,431662.0,0,0.884115
13800,TTD,1,436946.0,1,0.991076


Now we need to apply tfidf to the complaint documents to remove lowly weighted words such as the and focus only on important words such as 


In [2]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Simple class to store term and its importance score
class TermScore:
    def __init__(self, term, score):
        self.term = term
        self.score = score
    
    def __str__(self):
        # Format like: word(0.123)
        return f"{self.term}({self.score:.3f})"

def clean_text(text):
    """
    Clean up text by removing special characters and converting to lowercase
    """
    # Remove anything that's not a letter or space
    cleaned = re.sub(r'[^a-zA-Z ]+', '', text)
    return cleaned.lower().strip()

def clean_texts_in_dataframe(df, text_column='text'):
    """
    Clean all texts in a DataFrame column
    """
    if text_column not in df.columns:
        print(f"Error: '{text_column}' column not found!")
        return None
        
    # Make a copy so we don't change the original
    df_copy = df.copy()
    df_copy[text_column] = df_copy[text_column].apply(clean_text)
    return df_copy

class TextProcessor:
    """
    Main class to handle text processing using TF-IDF
    """
    def __init__(self, max_features=None):
        # Initialize the TF-IDF vectorizer
        self.vectorizer = TfidfVectorizer(
            max_features=max_features,
            stop_words='english',               
        
        )
        
    def calculate_tfidf(self, documents):
        # Convert documents to TF-IDF matrix
        tfidf_matrix = self.vectorizer.fit_transform(documents)
        
        # Get the words (features) that correspond to the matrix columns
        feature_names = self.vectorizer.get_feature_names_out()
        
        return {
            'matrix': tfidf_matrix.todense(),
            'words': feature_names
        }
    
    def process_documents(self, documents, max_words=100, show_scores=True):
        """
        Process documents and keep only the most important words based on TF-IDF
        """
        # First clean the texts
        cleaned_docs = [clean_text(doc) for doc in documents]
        
        # Calculate TF-IDF
        result = self.calculate_tfidf(cleaned_docs)
        matrix = result['matrix']
        words = result['words']
        
        processed_documents = []
        
        # Go through each document
        for doc_idx, doc_scores in enumerate(matrix):
            # Create a dictionary of word:score pairs
            word_scores = {}
            for word_idx, score in enumerate(doc_scores.T):
                word_scores[words[word_idx]] = float(score)
            
            # Sort words by score and keep top ones
            sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
            top_words = dict(sorted_words[:max_words])
            
            # Process original document words
            processed_words = []
            for word in cleaned_docs[doc_idx].split():
                if word in top_words:
                    if show_scores:
                        # Add the word with its score
                        term = TermScore(word, top_words[word])
                        processed_words.append(str(term))
                    else:
                        processed_words.append(word)
            
            processed_documents.append(' '.join(processed_words))
        
        return processed_documents
    
    def get_word_scores(self, result, doc_index, min_score=0.0):
        """
        Get all words and their importance scores for a specific document
        """
        doc_scores = result['matrix'][doc_index]
        
        # Create list of word-score pairs
        word_scores = []
        for word_idx, score in enumerate(doc_scores.T):
            score_value = float(score)
            if score_value >= min_score:
                word_scores.append(
                    TermScore(result['words'][word_idx], score_value)
                )
        
        # Sort by score (highest first)
        return sorted(word_scores, key=lambda x: x.score, reverse=True)

def process_file(input_file, output_file, text_column='text', max_words=100):
    """
    Process text data from a file and save results
    """
    try:
        # Read the file (works with CSV or TSV)
        if input_file.endswith('.tsv'):
            df = pd.read_csv(input_file, sep='\t')
        else:
            df = pd.read_csv(input_file)
        
        # Process the texts
        processor = TextProcessor()
        df = clean_texts_in_dataframe(df, text_column)
        texts = df[text_column].tolist()
        
        # Get TF-IDF scores and process
        processed_texts = processor.process_documents(texts, max_words)
        
        # Update and save
        df[text_column] = processed_texts
        df.to_csv(output_file, sep='\t', index=False)
        
        return df
        
    except FileNotFoundError:
        print(f"Error: Couldn't find file: {input_file}")
        return None

# Example usage / testing
if __name__ == "__main__":
    # Test with some example documents
    test_docs = [
        "I enjoy reading about Machine Learning and Machine Learning is my PhD subject",
        "I would enjoy a walk in the park",
        "I was reading in the library"
    ]
    
    # Create test DataFrame
    test_df = pd.DataFrame({"text": test_docs})
    
    # Create processor and process documents
    processor = TextProcessor()
    
    # Clean the texts
    cleaned_df = clean_texts_in_dataframe(test_df)
    cleaned_texts = cleaned_df["text"].tolist()
    
    # Process documents keeping top 5 words with scores
    processed_texts = processor.process_documents(
        cleaned_texts,
        max_words=5,
        show_scores=True
    )
    
    # Show results
    test_df["processed_text"] = processed_texts
    print("\nProcessed Texts with Scores:")
    print(test_df[["text", "processed_text"]])


Processed Texts with Scores:
                                                text  \
0  I enjoy reading about Machine Learning and Mac...   
1                   I would enjoy a walk in the park   
2                       I was reading in the library   

                                      processed_text  
0  enjoy(0.228) machine(0.599) learning(0.599) ma...  
1               enjoy(0.474) walk(0.623) park(0.623)  
2                      reading(0.605) library(0.796)  


/tmp/ipykernel_30/1118261806.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  word_scores[words[word_idx]] = float(score)


In [ ]:
#open complaint documents
file_path = '/kaggle/input/complaint-docs/complaintDocs_BertForClass.txt'
complaint_df = pd.read_csv(file_path, sep='\t', names=['docid', 'text', 'MotionResultCode'])
docs_df = pd.DataFrame(complaint_df, columns=['text'])

#process
processor = TextProcessor()

#clean 
cleaned_df = clean_texts_in_dataframe(docs_df)
cleaned_texts = cleaned_df["text"].tolist()

#process and keep top 5 words
processed_texts = processor.process_documents(
        cleaned_texts,
        max_words=5,
        show_scores=True
    )

#results
docs_df["processed_text"] = processed_texts
print("\nProcessed Texts with Scores:")
print(docs_df[["text", "processed_text"]])

In [12]:
#now we need to import the complaint documents to run tfidf 
file_path = '/kaggle/input/complaint-docs/complaintDocs_BertForClass.txt'
complaint_df = pd.read_csv(file_path, sep='\t', names=['docid', 'text', 'MotionResultCode'])
print(complaint_df)


        docid                                               text  \
0       docid                                               text   
1          77  Return Date: April 19, 2005 Superior Court Luc...   
2         453  RETURN DATE: MAY 10, 2005 SUPERIOR COURT MAURE...   
3         579  RETURN DATE: MAY 3, 2005 SUPERIOR COURT VERNON...   
4         669  1997 RETURN DATE: MAY 17, 2005 .. SUPERIOR COU...   
...       ...                                                ...   
8212  9989009  RETURN DATE: FEBRUARY 23, 2016 " SUPERIOR COUR...   
8213  9992499  RETURN DATE: FEBRUARY 16, 2016 : SUPERIOR COUR...   
8214  9996383  RET: RV DATE: FEBRUARY.2.2016 SUPERIOR COUR JE...   
8215  9999634  RETURN: FEBRUARY 23, 2016 SUPERIOR COURT LINDS...   
8216  9999747  RETURN DATE: FEBRUARY 16, 2016 SUPERIOR COURT ...   

      MotionResultCode  
0     MotionResultCode  
1                   OF  
2                   GR  
3                   OR  
4                   OR  
...                ...  
8212    

In [13]:
# we only want to apply these changes on column "text" so isolate column 


ctemp_df = pd.DataFrame(complaint_df, columns=['text'])
print(ctemp_df)

                                                   text
0                                                  text
1     Return Date: April 19, 2005 Superior Court Luc...
2     RETURN DATE: MAY 10, 2005 SUPERIOR COURT MAURE...
3     RETURN DATE: MAY 3, 2005 SUPERIOR COURT VERNON...
4     1997 RETURN DATE: MAY 17, 2005 .. SUPERIOR COU...
...                                                 ...
8212  RETURN DATE: FEBRUARY 23, 2016 " SUPERIOR COUR...
8213  RETURN DATE: FEBRUARY 16, 2016 : SUPERIOR COUR...
8214  RET: RV DATE: FEBRUARY.2.2016 SUPERIOR COUR JE...
8215  RETURN: FEBRUARY 23, 2016 SUPERIOR COURT LINDS...
8216  RETURN DATE: FEBRUARY 16, 2016 SUPERIOR COURT ...

[8217 rows x 1 columns]


In [14]:
ctemp_df = __normalizeDataset(ctemp_df)


print(ctemp_df)


                                                   text
0                                                  text
1     return date april   superior court lucille ame...
2     return date may   superior court maureen lebla...
3     return date may   superior court vernon skill ...
4     return date may    superior court bryce ridley...
...                                                 ...
8212  return date february    superior court  nicole...
8213  return date february    superior court kenya h...
8214  ret rv date february superior cour jeffrey hay...
8215  return february   superior court lindsay horny...
8216  return date february   superior court ausrele ...

[8217 rows x 1 columns]


In [23]:
t = ctemp_df["text"].tolist()
docs_matrix, feature_names = tfIdf(t)
tfidf_df = pd.DataFrame(docs_matrix, columns=feature_names)  # Each word (feature) becomes a column
res = mapTfIdfToDocs(docs_matrix, feature_names, t, 5)
ctemp_df["text"] = res


                                                   text  \
0                                                  text   
1     lucille bank bank bank bank bank bank bank ban...   
2     hersh premises premises premises agents servan...   
3     walkway walkway dangerous condition walkway da...   
4     suffield barbara suffield suffield suffield si...   
...                                                 ...   
8212  kuriansky premises premises premises trees tre...   
8213  parking lot namnoum asylum parking lot parking...   
8214  norwalk mark norwalk mark norwalk mark norwalk...   
8215  lindsay lindsay russo ventura ribeiro russo li...   
8216  gregory marie vehicle gregory vehicle marie ve...   

                                           tfidf_values  
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4


Processed Texts with TF-IDF Weights:
                                                text  \
0  I enjoy reading about Machine Learning and Mac...   
1                   I would enjoy a walk in the park   
2                       I was reading in the library   

                         processed_text_with_weights  
0  about(0.257) machine(0.514) learning(0.514) an...  
1  would(0.460) enjoy(0.349) walk(0.460) in(0.349...  
2  was(0.517) reading(0.394) in(0.394) the(0.394)...  

Detailed Term Weights for First Document:
learning: 0.514
machine: 0.514
about: 0.257
and: 0.257
is: 0.257
my: 0.257
phd: 0.257
subject: 0.257
enjoy: 0.195
reading: 0.195


/tmp/ipykernel_30/3511231108.py:169: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  weight_value = float(weight)


TFIDF VECTORIZATION complete


In [21]:
#now we assign numerical values to our 
ctemp_df["tfidf_values"] =.values.tolist() 


AttributeError: 'list' object has no attribute 'values'

In [20]:
print(tfidf_df)


            aa       aaa  aaaa  aab  aabf  aad  aae  aaf  aag  aai  ...   zs  \
0     0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
1     0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2     0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
3     0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
4     0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
...        ...       ...   ...  ...   ...  ...  ...  ...  ...  ...  ...  ...   
8212  0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
8213  0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
8214  0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
8215  0.030871  0.041583   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
8216  0.000000  0.000000   0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

       zu  zuboff  zucker  zuckerman  z

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, accuracy_score, auc, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Reshape, Concatenate


import sys
from sklearn.metrics import average_precision_score, accuracy_score, precision_recall_curve, roc_curve, roc_auc_score


In [63]:
X = filtered[["CaseMajorCode", "AttorneySpecialization", "CaseLocation", "MotionJurisNumber"]]
y = filtered[["MotionResultCode"]]



In [64]:
#neural embeddings (first layer of our neural network )

In [65]:
TARGET = "MotionResultCode" #this is what we are trying to predict
FEATURES = [col for col in filtered.columns if col != TARGET]

def neural_embedding(df):
    X = df.drop(TARGET, axis = 1)
    y = df[TARGET]
    
    #split the training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)
    #combine the x and y set together
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    #creating complete data set
    X_full = train[FEATURES]
    y_full = train[TARGET]
    X_test = test[FEATURES]
    y_test = test[TARGET]
    
    
    input_models = []
    embeddings = []
    for i in FEATURES:
        embed_name = i.replace(" ", "") + "__Embedding"
        num_unique = X_full[i].nunique()
        size_embed = int(min(np.ceil((num_unique)/2), 50))
        
        #creating input layer for neural network 
        input_mod = Input(shape=(1,), name= f"{num_unique}_input")
        output_mod = Embedding(input_dim=num_unique + 1, output_dim = size_embed, name = embed_name)(input_mod)
        #reshaps to one dimensional vector
        output_mod = Reshape(target_shape = (size_embed,))(output_mod)
        input_models.append(input_mod)
        embeddings.append(output_mod)
        
    final_output = Concatenate()(embeddings) if len(embeddings) > 1 else embeddings[0]
    model = Model(inputs= input_models, outputs=final_output)

    X_full_list = [X_full[feature].values for feature in FEATURES]
    X_test_list = [X_test[feature].values for feature in FEATURES]

    X_full_transformed = model.predict(X_full_list)
    X_test_transformed = model.predict(X_test_list)

    feature_names = [f'feature_{i}' for i in range(X_full_transformed.shape[1])]
    X_full_df = pd.DataFrame(X_full_transformed, columns=feature_names)
    X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names)

    return X_full_df, y_full, X_test_df, y_test
        
        
        
    
        
        
                          
                        
    
    



In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, y_train, X_test, y_test = neural_embedding(filtered)


InvalidArgumentError: Graph execution error:

Detected at node functional_5_1/MotionJurisNumber__Embedding_1/GatherV2 defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ke

In [68]:
pip install GPy

^C
Note: you may need to restart the kernel to use updated packages.


In [70]:
!pip install Gpy

ERROR: Could not find a version that satisfies the requirement Gpy (from versions: none)
ERROR: No matching distribution found for Gpy


In [67]:
#implement the training classifiers
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

import sys
from sklearn.metrics import average_precision_score, accuracy_score, precision_recall_curve, roc_curve, roc_auc_score

import numpy as np

import GPy # efficient Gaussian processes
from scipy.stats import beta

def evaluate_model(X_test, y_test, model, y_pred=None, pos_class=None):
    (fpr, tpr, roc_auc), (precision, recall, pr_auc), acc = evaluate(X_test, y_test, model, y_pred, pos_class)
    return dict(fpr=fpr, tpr=tpr, roc_auc=roc_auc, precision=precision, recall=recall, pr_auc=pr_auc, accuracy=acc)

def evaluate(X_test, y_test, clf, y_pred=None, pos_class=None):
    if y_pred is None:
        if hasattr(clf, 'predict_proba'):
            y_hat = clf.predict_proba(X_test) # predicting probability for each class
            pos_class = y_hat[:, 1]
        else:
            y_hat = clf.decision_function(X_test) # predicting confidence scores
            pos_class = y_hat
        y_pred = clf.predict(X_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = roc_curve(y_test.ravel(), pos_class)
    roc_auc = roc_auc_score(y_test, pos_class)

    precision = dict()
    recall = dict()
    pr_auc = dict()
    precision, recall, _ = precision_recall_curve(y_test.ravel(), pos_class)
    pr_auc = average_precision_score(y_test, pos_class)

    acc = accuracy_score(y_test, y_pred) 

    return (fpr, tpr, roc_auc), (precision, recall, pr_auc), acc

def runDecisionTreeClassifier(x_train,y_train,x_test,y_test):
    dt = tree.DecisionTreeClassifier()

    parameter_grid = {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [1, 2, 3, 4, 5],
        'max_features': list(range(1,len(x_train.columns)+1))
    }

    print("Grid search\n")
    grid_search = GridSearchCV(dt, param_grid=parameter_grid, cv=10)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    dt = grid_search.best_estimator_
    dt_predictions = dt.predict(x_test)
    dt_score = accuracy_score(y_test, dt_predictions) 
    print("decision tree classification accuracy on test data is " + str(dt_score), file=sys.stderr)

    random_sample = np.random.choice([0,1], len(y_test), replace=True)
    r_score = accuracy_score(y_test, random_sample) 
    print("random algorithm classification accuracy on test data is " + str(r_score), file=sys.stderr)
    print("feature importances " + str(dt.feature_importances_), file=sys.stderr)
    train_score = accuracy_score(y_train, dt.predict(x_train))
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)
    
    return evaluate_model(x_test, y_test, dt)

def runGradientBoostingClassifier(x_train,y_train,x_test,y_test):
    gb = GradientBoostingClassifier()

    parameter_grid = {
        "loss":["deviance","exponential"],
        "learning_rate": [0.01, 0.025],
        "min_samples_split": np.linspace(0.1, 0.5, 4),
        "min_samples_leaf": np.linspace(0.1, 0.5, 4),
        "max_depth":[3, 8],
        "max_features":["log2","sqrt",None],
        "subsample":[0.5, 0.618],
        "n_estimators":[10,25,50]
    }
    
    grid_search = GridSearchCV(gb, param_grid=parameter_grid, cv=10)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    gb = grid_search.best_estimator_
    
    gb_predictions = gb.predict(x_test)
    dt_score = accuracy_score(y_test, gb_predictions)
    print("accuracy score on test data: " +str(dt_score), file=sys.stderr)
    print("feature importances " + str(gb.feature_importances_), file=sys.stderr)
    train_score = accuracy_score(y_train, gb.predict(x_train)) 
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)
    
    return evaluate_model(x_test, y_test, gb)

def runAdaBoostClassifier(x_train,y_train,x_test,y_test):
    ab = AdaBoostClassifier()

    parameter_grid = {
        "base_estimator": [tree.DecisionTreeClassifier(), GradientBoostingClassifier(), ExtraTreesClassifier()],
        "learning_rate": [0.01, 0.025],
        "n_estimators":[10,25,50,100]
    }
    
    grid_search = GridSearchCV(ab, param_grid=parameter_grid, cv=10)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    ab = grid_search.best_estimator_
    
    ab_predictions = ab.predict(x_test)
    dt_score = accuracy_score(y_test, ab_predictions)
    print("accuracy score on test data: " +str(dt_score), file=sys.stderr)
    print("feature importances " + str(ab.feature_importances_), file=sys.stderr)
    train_score = accuracy_score(y_train, ab.predict(x_train)) 
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)
    
    return evaluate_model(x_test, y_test, ab)

def runExtraTreesClassifier(x_train,y_train,x_test,y_test):
    forest = ExtraTreesClassifier()

    parameter_grid = {
        "criterion": ["gini",  "entropy"],
        "n_estimators":[10,50,100,250,500],
        "max_depth":[3,5,8,None],
        "min_samples_split": np.linspace(0.1, 0.5, 12),
        "min_samples_leaf": np.linspace(0.1, 0.5, 12),
        "max_features":["log2","sqrt",None]
    }
    
    grid_search = GridSearchCV(forest, param_grid=parameter_grid, cv=10)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    forest = grid_search.best_estimator_
    
    importances = forest.feature_importances_
    print(importances, file=sys.stderr)
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    print("Feature ranking:", file=sys.stderr)

    for f in range(x_train.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]), file=sys.stderr)

    etc_predictions = forest.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, forest.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)
    return evaluate_model(x_test, y_test, forest)

def runStochasticGradientDescentClassifier(x_train,y_train,x_test,y_test):
    sgd = SGDClassifier()
    
    parameter_grid = {
        "loss": ['log','modified_huber','squared_hinge'],
        "penalty":['l1', 'l2']

        # "alpha":[],
        # "fit_intercept": [],
        # "max_iter":[],
        # "tol": [],
        # "shuffle": [],
        # "verbose": [],
        # "epsilon": [],
        # "n_jobs": [],
        # "random_state": [],
        # "eta0": [],
        # "power_t": [],
        # "early_stopping": [],
        # "validation_fraction": [],
        # "n_iter_no_change": [],
        # "class_weight": [],
        # "warm_start": [],
        # "average": []
    
    }
    
    grid_search = GridSearchCV(sgd, param_grid=parameter_grid, cv=10)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    sgd = grid_search.best_estimator_

    print("Feature ranking:", file=sys.stderr)

    etc_predictions = sgd.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, sgd.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)
    return evaluate_model(x_test, y_test, sgd)

def runKNeighborsClassifier(x_train,y_train,x_test,y_test):
    kNC = KNeighborsClassifier()

    parameter_grid = {
        "weights": ["uniform","distance"],
        "algorithm":["brute","ball_tree","kd_tree","auto"],
        #"n_jobs":[-1,10,50]
    }
    
    grid_search = GridSearchCV(kNC, param_grid=parameter_grid, cv=5)
    grid_search.fit(x_train, y_train)
    print('Best score: {}'.format(grid_search.best_score_), file=sys.stderr)
    print('Best parameters: {}'.format(grid_search.best_params_), file=sys.stderr)

    kNC = grid_search.best_estimator_

    etc_predictions = kNC.predict(x_test)
    dt_score =accuracy_score(y_test, etc_predictions)
    print("accuracy score on test data: " + str(dt_score), file=sys.stderr)
    train_score = accuracy_score(y_train, kNC.predict(x_train))
    print("accuracy score on training data: " + str( train_score ), file=sys.stderr)    
    return evaluate_model(x_test, y_test, kNC)

def runSparseGaussianProcessClassifier(x_train,y_train,x_test,y_test, num_inducing=10, seed=42, optimize=True, plot=False):
    x_train = x_train.to_numpy()
    y_train = y_train.to_numpy()[:,np.newaxis]
    x_test = x_test.to_numpy()
    y_test = y_test.to_numpy()[:,np.newaxis]
    m = GPy.models.SparseGPClassification(x_train, y_train, num_inducing=num_inducing)

    if optimize:
        m.optimize()
        
    if plot:
        from matplotlib import pyplot as plt
        fig, axes = plt.subplots(2, 1)
        m.plot_f(ax=axes[0])
        m.plot(ax=axes[1])

    print(m)
    
    gp_predictions = m.predict(x_test)[0]
    gp_predictions_soft = m.predict(x_test)[0]
    print("test set predictions: "+ str(gp_predictions))
    gp_predictions[gp_predictions>=0.5]=1
    gp_predictions[gp_predictions<0.5]=0
    gp_score = accuracy_score(y_test, gp_predictions)
    print("accuracy score on test data: " +str(gp_score), file=sys.stderr)

    gp_predictions_train = m.predict(x_train)[0]
    gp_predictions_train[gp_predictions_train>=0.5]=1
    gp_predictions_train[gp_predictions_train<0.5]=0
    train_score = accuracy_score(y_train, gp_predictions_train)
    print("accuracy score on training data: " + str(train_score), file=sys.stderr)
    return evaluate_model(x_test, y_test, m, gp_predictions, gp_predictions_soft)

ModuleNotFoundError: No module named 'GPy'